In [ ]:
import os
import random
import numpy as np
from io import BytesIO
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensrflow as tf

In [2]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [22]:
BASE_DIR = r'C:\Users\toran\Documents\GitHub\ML_PadiCare\data'
train_dir = os.path.join(BASE_DIR, 'train')
validation_dir = os.path.join(BASE_DIR, 'validation')

In [23]:
validation_datagen = ImageDataGenerator(rescale=1./255)

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
train_dataset = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)
validation_dataset = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)


Found 1800 images belonging to 4 classes.
Found 452 images belonging to 4 classes.


In [24]:
base_model = tf.keras.applications.VGG16(
    weights='imagenet', include_top=False, input_shape=(150, 150, 3)
)
base_model.trainable = False

In [25]:
model = tf.keras.models.Sequential([
    base_model,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(4, activation='softmax')
])

In [26]:
base_model.trainable = True
for layer in base_model.layers[:-4]:
    layer.trainable = False

In [27]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [28]:
history = model.fit(
    train_dataset,
    epochs=20,
    validation_data=validation_dataset,
    verbose=2,
    callbacks=[tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5)]
)

C:\Users\toran\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20


C:\Users\toran\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


57/57 - 226s - 4s/step - accuracy: 0.3167 - loss: 2.1247 - val_accuracy: 0.2898 - val_loss: 1.6521 - learning_rate: 1.0000e-04
Epoch 2/20
57/57 - 212s - 4s/step - accuracy: 0.4006 - loss: 1.8324 - val_accuracy: 0.2367 - val_loss: 1.9000 - learning_rate: 1.0000e-04
Epoch 3/20
57/57 - 189s - 3s/step - accuracy: 0.4228 - loss: 1.6937 - val_accuracy: 0.2500 - val_loss: 3.7907 - learning_rate: 1.0000e-04
Epoch 4/20
57/57 - 188s - 3s/step - accuracy: 0.4383 - loss: 1.6129 - val_accuracy: 0.2323 - val_loss: 3.0396 - learning_rate: 1.0000e-04
Epoch 5/20
57/57 - 193s - 3s/step - accuracy: 0.4778 - loss: 1.4552 - val_accuracy: 0.2854 - val_loss: 3.5444 - learning_rate: 1.0000e-04
Epoch 6/20
57/57 - 185s - 3s/step - accuracy: 0.4728 - loss: 1.4173 - val_accuracy: 0.2810 - val_loss: 3.5475 - learning_rate: 1.0000e-04
Epoch 7/20
57/57 - 198s - 3s/step - accuracy: 0.4928 - loss: 1.3809 - val_accuracy: 0.2456 - val_loss: 2.6910 - learning_rate: 2.0000e-05
Epoch 8/20
57/57 - 189s - 3s/step - accuracy:

In [ ]:
# File uploader for prediction
uploader = widgets.FileUpload(accept="image/*", multiple=True)
display(uploader)

# Output widget
out = widgets.Output()
display(out)

# Function to process and predict each uploaded image
def file_predict(filename, file, out):
    """Make predictions and display results."""
    # Load image and preprocess
    image = tf.keras.utils.load_img(file, target_size=(150, 150))
    image = tf.keras.utils.img_to_array(image)
    image = np.expand_dims(image, axis=0) / 255.0  # Normalize

    # Predict
    predictions = model.predict(image, verbose=0)
    predicted_class = np.argmax(predictions, axis=1)[0]

    # Map class indices to labels
    class_names = train_dataset.class_indices
    class_names = {v: k for k, v in class_names.items()}  # Invert mapping
    predicted_label = class_names[predicted_class]

    # Display results
    with out:
        print(f"{filename} -> Predicted: {predicted_label}")

FileUpload(value=(), accept='image/*', description='Upload', multiple=True)

Output()

: 